In [1]:
from __future__ import division
import pickle
import numpy as np
import itertools
# from matplotlib import pyplot as plt

import sys
# path_to_codebase = '/mnt/Study/umass/sem3/maxEnt/src/codebase/'
path_to_codebase = './codebase/'
sys.path.insert(0, path_to_codebase)
from codebase.utils import load_disease_data, load_disease_data_perturb
from codebase.extract_features import ExtractFeatures
from codebase.optimizer import Optimizer
from codebase.robust_optimizer import RobustOptimizer


In [19]:
filePath = '../data/Age50_DataExtract_fy.csv'
# filePath = '../data/2010-2014-fy.csv'
# filePath = '../data/test1-fy.csv'

entropy_estimator = 'JAMES-STEIN'
k_val = 4
eps_probs = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [3]:
# Creating the 2-way constraint dict from MBA analysis
two_wayc = {}
keys = [ (0,1), (3,6), (3,4), (1,4)]

common_val = (1,1)  # Since MBA only looks for (1,1) pairs
for k in keys:
    two_wayc[k] = common_val


# Creating the 3-way constraint dict from MBA analysis
three_wayc = {}
keys = [(0,1,4), (3,6,4)]  

common_val = (1,1,1)  # Since MBA only looks for (1,1,1) pairs
for k in keys:
    three_wayc[k] = common_val


# Creating the 4-way constraint dict from MBA analysis
four_wayc = {}
keys = [(0,1,3,4)]

common_val = (1,1,1,1)  # Since MBA only looks for (1,1,1,1) pairs 
for k in keys:
    four_wayc[k] = common_val




In [4]:
data_array = load_disease_data(filePath)


In [5]:
data_array.shape

(713, 9)

In [6]:
feats = ExtractFeatures(data_array, entropy_estimator, k_val)

feats.set_two_way_constraints(two_wayc)
feats.set_three_way_constraints(three_wayc)
feats.set_four_way_constraints(four_wayc)
# feats.compute_topK_feats()
feats.partition_features()



Creating the feature graph
('Added edge for:', (0, 1))
('Added edge for:', (3, 4))
('Added edge for:', (1, 4))
('Added edge for:', (3, 6))
('Added edge for:', (3, 6, 4))
('Added edge for:', (0, 1, 4))
('Added edge for:', (3, 6, 4))
('Added edge for:', (0, 1, 4))
Partioning the feature graph
Finding the connected components


In [7]:
opt_unreg = Optimizer(feats) 
soln_opt_unreg = opt_unreg.solver_optimize()
soln_opt_unreg

([(array([-2.43980609e+00, -2.42867115e+00, -4.86012077e+00, -2.30412140e+00,
          -5.75680854e+00,  1.50358353e+00,  4.34750116e+00,  2.35774735e+00,
           2.30370847e+00,  8.09818411e-04, -1.22649910e+01, -2.69221654e+00]),
   776.2383343857186,
   {'funcalls': 975,
    'grad': array([-0.00245564, -0.00089813, -0.00172804,  0.00104592, -0.00101181,
           -0.00159162, -0.00108002, -0.00195541, -0.00051159, -0.00013642,
            0.        ,  0.        ]),
    'nit': 70,
    'task': 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH',
    'warnflag': 0}),
  [[-2.405569907551479]],
  [[-3.4011973816621555]],
  [[-2.5847519847577165]],
  [[-3.1606620876360765]]],
 [1.3632009093955928,
  1.0902140672782874,
  1.0333333333333334,
  1.0754147812971342,
  1.0423976608187135])

In [8]:
opt_reg = RobustOptimizer(feats, 2.0) 
soln_opt_reg = opt_reg.solver_optimize()
soln_opt_reg

([(array([-2.43547013, -2.41909371, -4.78321724, -2.30009341, -5.51534756,
           1.48071796,  3.97391347,  2.25902105,  2.01881528,  0.36272709,
          -2.69161499, -0.53605307]),
   781.595584914046,
   {'funcalls': 1014,
    'grad': array([ 4.85442797e-03,  5.65023583e-03,  1.90993887e-03,  3.41060513e-05,
            7.95807864e-04,  2.80806489e-03,  3.41060513e-04, -1.61435310e-03,
           -1.06865627e-03, -1.54614099e-03,  9.66338121e-04, -7.84439180e-04]),
    'nit': 70,
    'task': 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH',
    'warnflag': 0}),
  [[-2.405569907551479]],
  [[-3.4011973816621555]],
  [[-2.5847519847577165]],
  [[-3.1606620876360765]]],
 [1.3661097710046246,
  1.0902140672782874,
  1.0333333333333334,
  1.0754147812971342,
  1.0423976608187135])

In [9]:
def kldiv_emp_mxe(optobj, data):
#     data = optobj.feats_obj.data_arr
    num_feats = data.shape[1]
    all_perms = itertools.product([0, 1], repeat=num_feats)
    kl_sum = 0.0
    for tmp in all_perms:
        vec = np.asarray(tmp)        
        emp_count = 0
        for datavec in data:
            if sum(datavec == vec) == num_feats:
                emp_count += 1
        
        if emp_count == 0:
            kl_sum += 0
        else:
            mxe_prob = optobj.prob_dist(vec)
            emp_prob = emp_count * 1.0 / data.shape[0]
            kl_sum += emp_prob * np.log(emp_prob/mxe_prob)
    
    return kl_sum
            
    
    

In [10]:
from collections import defaultdict

def kldiv_2(optobj, data):
#     data = optobj.feats_obj.data_arr
    num_feats = data.shape[1]
    
    count_dict = defaultdict(int)
    for datavec in data:
        tup = tuple(datavec)
        count_dict[tup] += 1        
    
    kl_sum = 0.0
    for tup in count_dict.keys():
        vec = np.asarray(tup)
        count = count_dict[tup]
        
        if count == 0:
            kl_sum += 0
        else:
            emp_prob = count * 1.0 / data.shape[0]
            mxe_prob = optobj.prob_dist(vec)
            kl_sum += emp_prob * np.log(emp_prob/mxe_prob)   
    
    return kl_sum

In [11]:
data = opt_reg.feats_obj.data_arr

In [22]:
perturb_data = load_disease_data_perturb(filePath, 0.1)

In [13]:
perturb_data.shape

(713, 9)

In [23]:
kldiv_2(opt_unreg, perturb_data)

0.92420359791924

In [24]:
kldiv_2(opt_reg, perturb_data)

0.8650163825073218

In [25]:
unreg_kldict = {}
reg_kldict = {}
for e in eps_probs:
    perturb_data = load_disease_data_perturb(filePath, e)
    unreg_kldict[e] = kldiv_2(opt_unreg, perturb_data)
    reg_kldict[e] = kldiv_2(opt_reg, perturb_data)

In [28]:
unreg_kldict # un-regularized model

{0.05: 0.40564122582720263,
 0.1: 0.9462314308161917,
 0.15: 1.6773523876629757,
 0.2: 2.57519810559319,
 0.25: 3.558478908154694,
 0.3: 4.017281782674745,
 0.4: 6.690886919488896,
 0.5: 9.46288961063854,
 0.6: 12.758250916490354,
 0.7: 16.586892604394066,
 0.8: 21.20588578086693,
 0.9: 26.957309582618834}

In [29]:
reg_kldict # regularized model

{0.05: 0.3677458919349196,
 0.1: 0.84061220600919,
 0.15: 1.5227344243186272,
 0.2: 2.35381965081224,
 0.25: 3.178909005088776,
 0.3: 3.56132318325954,
 0.4: 5.741276792274964,
 0.5: 7.678114714760526,
 0.6: 10.066709820210026,
 0.7: 12.328817270950031,
 0.8: 15.126934410131774,
 0.9: 18.267422853971745}

In [ ]:


# opt.compare_marginals()

# m2, e2 = opt.compare_constraints_2way()
# # print m2
# # print e2

# m3, e3 = opt.compare_constraints_3way()
# # print m3
# # print e3

# m4, e4 = opt.compare_constraints_4way()
# # print m4
# # print e4

# #### PLOTS #### 
# num_feats = data_array.shape[1]
# all_perms = itertools.product([0, 1], repeat=num_feats)

# mxt_prob = np.zeros(num_feats + 1)
# for tmp in all_perms:
#     vec = np.asarray(tmp)
#     j = sum(vec)
#     mxt_prob[j] += opt.prob_dist(vec)

# emp_prob = np.zeros(num_feats + 1)
# for vec in data_array:
#     j = sum(vec)
#     emp_prob[j] += 1
# emp_prob /= data_array.shape[0] # N

# print mxt_prob, emp_prob

# xvec = [i+1 for i in range(num_feats + 1)]
# x_ticks = np.arange(0, num_feats+2, 1.0)
# plot_lims = [0,  num_feats+2, -0.1, 1.0]
# # Both on same plot
# plt.figure()
# plt.plot(xvec, emp_prob, 'ro', label='empirical')  # empirical
# plt.plot(xvec, mxt_prob, 'bo', label='maxent')  # maxent
# plt.legend()
# plt.xticks(x_ticks)
# plt.axis(plot_lims)
# fname_plt = '../out/plot_toy_mba_e-' + str('%.3f' %eps_prob) + '.png'
# plt.savefig(fname_plt)

